# **Modélisation statistique**
Nous allons mener des régressions linéaires de la production électrique éolienne (données API RTE) sur les données météorologiques de vents (données API ...)

Dans un premier temps, nous allons créer un data frame qui contiendra la moyenne nationale de la production électrique éolienne ainsi que la moyenne des vents nationaux, heure par heure pour l'année 2021.
Ensuite, nous affinerons l'analyse en faisant des régressions par région/année.

### **1) Récupération des données**

Dans un premier temps, nous cherchons à mettre les dates des données météorologique et des données RTE dans le même format. Le format cherché est, par exemple pour le 5 mai 2020 à 12h:

**2020-05-05:12:00**

#### **a) Données météorologiques**

In [1]:
import pandas as pd

In [2]:
donnees_vent = pd.read_csv(r'C:\Users\peign\Desktop\ProjetPython\donnees_vent.csv', sep=",", names=['Index', 'Time', 'Windspeed'])

In [3]:
donnees_vent.head()

,Index,Time,Windspeed
0,0,"DatetimeIndex(['2011-04-23 00:00:00', '2011-04...","[2.34, 2.33, 1.8, 1.84, 1.7, 2.01, 1.35, 1.08,..."
1,1,"DatetimeIndex(['2014-04-01 00:00:00', '2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
2,2,"DatetimeIndex(['2014-04-01 00:00:00', '2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
3,3,"DatetimeIndex(['2014-04-01 00:00:00', '2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
4,4,"DatetimeIndex(['2021-11-04 00:00:00', '2021-11...","[1.61, 1.71, 1.53, 1.86, 1.82, 1.57, 1.61, 1.7..."


Le format des données est le suivant:

-L'index représente le numéro des éoliennes, numérotées de 0 à XXXX.

-La variable "Time" contient la liste de toutes les dates auxquelles on observe la vitesse du vent à la position de l'éolienne.

-La variable "Windspeed" contient la liste de la vitesse des vents.

Dans un premier temps, on va chercher à "nettoyer" la variable "Time" afin de créer une liste de dates harmonisées.

In [4]:
donnees_vent["Time"][0]

"DatetimeIndex(['2011-04-23 00:00:00', '2011-04-23 01:00:00',\n               '2011-04-23 02:00:00', '2011-04-23 03:00:00',\n               '2011-04-23 04:00:00', '2011-04-23 05:00:00',\n               '2011-04-23 06:00:00', '2011-04-23 07:00:00',\n               '2011-04-23 08:00:00', '2011-04-23 09:00:00',\n               ...\n               '2022-12-04 14:00:00', '2022-12-04 15:00:00',\n               '2022-12-04 16:00:00', '2022-12-04 17:00:00',\n               '2022-12-04 18:00:00', '2022-12-04 19:00:00',\n               '2022-12-04 20:00:00', '2022-12-04 21:00:00',\n               '2022-12-04 22:00:00', '2022-12-04 23:00:00'],\n              dtype='datetime64[ns]', length=101856, freq=None)"

In [5]:
caracteres_time = ["\n","DatetimeIndex(", "               "," ","...","'","[ns]"]
    
for string in caracteres_time:
    donnees_vent["Time"] = donnees_vent["Time"].apply(lambda x: x.replace(string, ''))
    
donnees_vent["Time"] = donnees_vent["Time"].str.extract(r"\[(.+?)\]")
donnees_vent["Time"] = donnees_vent["Time"].str.replace(',', ' ')

In [6]:
caracteres_vent = ["[", "]", " "]

for string in caracteres_vent:
    donnees_vent["Windspeed"] = donnees_vent["Windspeed"].apply(lambda x: x.replace(string, ''))

donnees_vent["Windspeed"] = donnees_vent["Windspeed"].str.replace(',', ' ')

In [7]:
donnees_vent.head()

,Index,Time,Windspeed
0,0,2011-04-2300:00:00 2011-04-2301:00:00 2011-04-...,2.34 2.33 1.8 1.84 1.7 2.01 1.35 1.08 0.92 1.0...
1,1,2014-04-0100:00:00 2014-04-0101:00:00 2014-04-...,0.86 1.2 1.66 1.21 1.51 2.02 2.51 2.81 2.73 2....
2,2,2014-04-0100:00:00 2014-04-0101:00:00 2014-04-...,0.86 1.2 1.66 1.21 1.51 2.02 2.51 2.81 2.73 2....
3,3,2014-04-0100:00:00 2014-04-0101:00:00 2014-04-...,0.86 1.2 1.66 1.21 1.51 2.02 2.51 2.81 2.73 2....
4,4,2021-11-0400:00:00 2021-11-0401:00:00 2021-11-...,1.61 1.71 1.53 1.86 1.82 1.57 1.61 1.75 1.44 0...


Maintenant que les variables "Time" et "Windspeed" sont nettoyées, on va utiliser la fonction explode afin d'avoir une base de données "classique" pour pouvoir faire des régressions. Nous allons étendre les listes afin que chaque vent soit associé à une date précise.

Premièrement, on convertit nos données considérés comme "string" en liste.

In [8]:
donnees_vent["Time"] = donnees_vent["Time"].str.split(' ')
donnees_vent["Time"] = donnees_vent["Time"].tolist()

In [ ]:
#Ce code fait planter mon ordi
donnees_vent["Windspeed"] = donnees_vent["Windspeed"].str.split(' ')
donnees_vent["Windspeed"] = donnees_vent["Windspeed"].tolist()

C:\Users\peign\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
donnees_vent.head()

,Index,Time,Windspeed
0,0,"[2011-04-2300:00:00, 2011-04-2301:00:00, 2011-...",2.34 2.33 1.8 1.84 1.7 2.01 1.35 1.08 0.92 1.0...
1,1,"[2014-04-0100:00:00, 2014-04-0101:00:00, 2014-...",0.86 1.2 1.66 1.21 1.51 2.02 2.51 2.81 2.73 2....
2,2,"[2014-04-0100:00:00, 2014-04-0101:00:00, 2014-...",0.86 1.2 1.66 1.21 1.51 2.02 2.51 2.81 2.73 2....
3,3,"[2014-04-0100:00:00, 2014-04-0101:00:00, 2014-...",0.86 1.2 1.66 1.21 1.51 2.02 2.51 2.81 2.73 2....
4,4,"[2021-11-0400:00:00, 2021-11-0401:00:00, 2021-...",1.61 1.71 1.53 1.86 1.82 1.57 1.61 1.75 1.44 0...


#### **b) Données RTE**
On importe les données RTE auparavant obtenu grâce à une API.

**Pour l'instant:** données test de Janvier à Février 2020

In [146]:
data_RTE = pd.read_csv('RTE_test.csv')

In [147]:
data_RTE.head()

,Unnamed: 0,start_date,end_date,updated_date,value
0,0,2020-01-01 00:00:00+01:00,2020-01-01 01:00:00+01:00,2020-01-01T00:37:05+01:00,3361
1,1,2020-01-01 01:00:00+01:00,2020-01-01 02:00:00+01:00,2020-01-01T01:37:08+01:00,2810
2,2,2020-01-01 02:00:00+01:00,2020-01-01 03:00:00+01:00,2020-01-01T02:37:06+01:00,2655
3,3,2020-01-01 03:00:00+01:00,2020-01-01 04:00:00+01:00,2020-01-01T03:37:07+01:00,2736
4,4,2020-01-01 04:00:00+01:00,2020-01-01 05:00:00+01:00,2020-01-01T04:37:05+01:00,2631


On renomme les trois colonnes nécessaires et on enlève les colonnes inutiles

In [148]:
columns_to_drop = ['end_date', 'updated_date']
data_RTE = data_RTE.drop(columns_to_drop, axis=1)

columns_to_rename = {'Unnamed: 0': 'Index', 'start_date': 'Time', 'value': 'Electricity'}
data_RTE = data_RTE.rename(columns=columns_to_rename)

On enlève les éléments de la date inutile afin d'harmoniser

In [149]:
caracteres = ["+01:00"," "]
    
for string in caracteres:
    data_RTE["Time"] = data_RTE["Time"].apply(lambda x: x.replace(string, ''))

In [150]:
data_RTE.head()

,Index,Time,Electricity
0,0,2020-01-0100:00:00,3361
1,1,2020-01-0101:00:00,2810
2,2,2020-01-0102:00:00,2655
3,3,2020-01-0103:00:00,2736
4,4,2020-01-0104:00:00,2631


Les dates sont bien au format harmonisé voulu.

### **2) Régression au niveau national**
#### **Calcul des moyennes**

In [ ]:
hours = df.groupby(df.index.hour)

mean_nat = hours.mean()

#### **Régression linéaire**

In [ ]:
import statsmodels.formula.api as smf

def regression_results(data):
    model = smf.ols("electricity ~ wind", data=data).fit()
    predictions = model.predict(data)
    print(model.summary())
    
regression_results(mean_nat)  

#### **Graphiques**

In [ ]:
def plot_linear_regression(data):
    plot = sns.regplot(x="wind", y="electricity", data=data)
    sns.lmplot(x="wind", y="electricity", data=data)
    return plot

plot = plot_linear_regression(mean_nat)
plt.show()